In [35]:
import pandas as pd
import numpy as np
import yfinance as yf
from datetime import datetime
import os

In [36]:
# Define tickers (Yahoo Finance symbols)
tickers = {
    "Nifty50": "^NSEI",
    "Nifty100": "^CNX100",
    "Nifty200": "^CNX200",
    "Nifty500": "^CRSLDX"  # alternative: "^CNX500"
}

# Function to flatten multi-index columns
def flatten_columns(df):
    if isinstance(df.columns, pd.MultiIndex):
        df.columns = ['_'.join([str(c) for c in col if c and c != 'CL=F']).strip('_') for col in df.columns]
    return df

# Function to fetch and clean a ticker's data
def fetch_and_clean(ticker_symbol, name):
    df = yf.download(ticker_symbol, start="2013-01-01", group_by="ticker", auto_adjust=False)
    df = flatten_columns(df)
    df = df.reset_index()  # Make 'Date' a column
    df['Date'] = pd.to_datetime(df['Date'])  # Ensure proper datetime
    df["Ticker"] = name
    return df

# Fetch all tickers into individual DataFrames
df_nifty50 = fetch_and_clean(tickers["Nifty50"], "Nifty50")
df_nifty100 = fetch_and_clean(tickers["Nifty100"], "Nifty100")
df_nifty200 = fetch_and_clean(tickers["Nifty200"], "Nifty200")
df_nifty500 = fetch_and_clean(tickers["Nifty500"], "Nifty500")

# (Optional) Store in dictionary for easier programmatic use
dfs = {
    "Nifty50": df_nifty50,
    "Nifty100": df_nifty100,
    "Nifty200": df_nifty200,
    "Nifty500": df_nifty500
}

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [37]:
df_wti.head()  # Display the first few rows of the WTI DataFrame for verification

,Date,Open,High,Low,Close,Adj Close,Volume,Ticker
0,2013-01-02,91.779999,93.870003,91.559998,93.120003,93.120003,203865,WTI
1,2013-01-03,92.910004,93.300003,92.489998,92.919998,92.919998,189812,WTI
2,2013-01-04,92.860001,93.209999,91.519997,93.089996,93.089996,210747,WTI
3,2013-01-07,93.209999,93.349998,92.419998,93.190002,93.190002,166292,WTI
4,2013-01-08,93.320000,93.800003,92.669998,93.150002,93.150002,195867,WTI


In [38]:
output_folder = r"C:\Users\Stevi\OneDrive\Documents\Projects\Crude-Oil\Data ingestion"  # replace with your actual folder path
os.makedirs(output_folder, exist_ok=True)  # create the folder if it doesn't exist

for name, df in dfs.items():
    file_path = os.path.join(output_folder, f"{name}.csv")
    df.to_csv(file_path, index=False)

In [39]:
# Date range
start_date = "2024-01-01"
end_date = "2025-07-15"

# Output folder
output_dir = r"C:\Users\Stevi\OneDrive\Documents\Projects\Crude-Oil\Data ingestion"
os.makedirs(output_dir, exist_ok=True)

# Download data
wti = yf.download("CL=F", start=start_date, end=end_date)
brent = yf.download("BZ=F", start=start_date, end=end_date)
usd_inr = yf.download("INR=X", start=start_date, end=end_date)

# Prepare USDINR Close column
usd_inr = usd_inr[["Close"]].rename(columns={"Close": "USDINR"})

# Robust conversion function
def convert_to_inr(df_usd, fx_df):
    df = df_usd.copy()
    
    # Merge on date index
    df_merged = df.merge(fx_df, how="inner", left_index=True, right_index=True)
    
    usd_to_inr = df_merged["USDINR"].values  # get values to avoid index alignment issues
    
    # Multiply only numeric columns
    numeric_cols = df_merged.select_dtypes(include='number').columns.drop("USDINR")
    for col in numeric_cols:
        df_merged[col] = (df_merged[col].values * usd_to_inr).round(2)
    
    return df_merged.drop(columns=["USDINR"])

# After conversion
wti_inr = convert_to_inr(wti, usd_inr)
brent_inr = convert_to_inr(brent, usd_inr)

# Flatten column headers if needed
for df in [wti_inr, brent_inr]:
    if isinstance(df.columns[0], tuple):  # MultiIndex check
        df.columns = [col[0] for col in df.columns]

# Save to CSV
wti_inr.to_csv(os.path.join(output_dir, "wti_inr.csv"), index=True)
brent_inr.to_csv(os.path.join(output_dir, "brent_inr.csv"), index=True)

print("✅ Data saved with clean headers.")

C:\Users\Stevi\AppData\Local\Temp\ipykernel_10040\2361206918.py:10: FutureWarning: YF.download() has changed argument auto_adjust default to True
  wti = yf.download("CL=F", start=start_date, end=end_date)
[*********************100%***********************]  1 of 1 completed


C:\Users\Stevi\AppData\Local\Temp\ipykernel_10040\2361206918.py:11: FutureWarning: YF.download() has changed argument auto_adjust default to True
  brent = yf.download("BZ=F", start=start_date, end=end_date)
[*********************100%***********************]  1 of 1 completed
C:\Users\Stevi\AppData\Local\Temp\ipykernel_10040\2361206918.py:12: FutureWarning: YF.download() has changed argument auto_adjust default to True
  usd_inr = yf.download("INR=X", start=start_date, end=end_date)
[*********************100%***********************]  1 of 1 completed

✅ Data saved with clean headers.



C:\Users\Stevi\AppData\Local\Temp\ipykernel_10040\2361206918.py:27: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  numeric_cols = df_merged.select_dtypes(include='number').columns.drop("USDINR")
C:\Users\Stevi\AppData\Local\Temp\ipykernel_10040\2361206918.py:31: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  return df_merged.drop(columns=["USDINR"])
C:\Users\Stevi\AppData\Local\Temp\ipykernel_10040\2361206918.py:27: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  numeric_cols = df_merged.select_dtypes(include='number').columns.drop("USDINR")
C:\Users\Stevi\AppData\Local\Temp\ipykernel_10040\2361206918.py:31: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  return df_merged.drop(columns=["USDINR"])
